# Zindi Project - Financial Inclusion in Africa

Financial inclusion remains one of the main obstacles to economic and human development in Africa. For example, across Kenya, Rwanda, Tanzania, and Uganda only 9.1 million adults (or 14% of adults) have access to or use a commercial bank account.

Traditionally, access to bank accounts has been regarded as an indicator of financial inclusion. Despite the proliferation of mobile money in Africa, and the growth of innovative fintech solutions, banks still play a pivotal role in facilitating access to financial services. Access to bank accounts enable households to save and make payments while also helping businesses build up their credit-worthiness and improve their access to loans, insurance, and related services. Therefore, access to bank accounts is an essential contributor to long-term economic growth.

__The objective of this project is to create a machine learning model to predict which individuals are most likely to have or use a bank account.__ The models and solutions developed can provide an indication of the state of financial inclusion in Kenya, Rwanda, Tanzania and Uganda, while providing insights into some of the key factors driving individuals’ financial security.

# Our Goal

Our Goal is to predict values for our NaNs in our target column bank_account. 

## Data Overview

| column | additional information |
|--------|------------------------|
| country | Country interviewee is in |
| year | Year survey was done in  |
| uniqueid | Unique identifier for each interviewee | 
| location_type | Type of location: Rural, Urban |
| cellphone_access | If interviewee has access to a cellphone: Yes, No |
| household_size | Number of people living in one house |
| age_of_respondent | The age of the interviewee |
| gender_of_respondent | Gender of interviewee: Male, Female | 
| relationship_with_head | The interviewee’s relationship with the head of the house:Head of Household, Spouse, Child, Parent, Other relative, Other non-relatives, Dont know |
| marital_status | The martial status of the interviewee: Married/Living together, Divorced/Seperated, Widowed, Single/Never Married, Don’t know |
| education_level | Highest level of education: No formal education, Primary education, Secondary education, Vocational/Specialised training, Tertiary education, Other/Dont know/RTA |
| job_type | Type of job interviewee has: Farming and Fishing, Self employed, Formally employed Government, Formally employed Private, Informally employed, Remittance Dependent, Government Dependent, Other Income, No Income, Dont Know/Refuse to answer |

Importing Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import chi2_contingency
from matplotlib.ticker import PercentFormatter
from sklearn.model_selection import train_test_split, GridSearchCV
import imblearn
plt.rcParams["patch.force_edgecolor"] = True
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from scipy import stats

Importing the data

In [ ]:
# Import the data into a dataframe
test = pd.read_csv('data/Test.csv')
train = pd.read_csv('data/train.csv')

# Make a new Dataframe with all the data
df = pd.concat([test, train])

In [ ]:
# Export the dataframe into a csv:

# Define the path to the folder in your repository
folder_path = 'data/'

# Define the file name and extension
file_name = 'data.csv'

# Concatenate the folder path and file name
file_path = f'{folder_path}/{file_name}'

# Export the DataFrame to the specified folder
df.to_csv(file_path, index=False)


## EDA: Exploring the data

In this part of the notebook we look and analyze our financial inclusion data we got from Zindi.

In [ ]:
# Print the shape of the data
print('Financial Inclusion dataset')
print('==================')
print('# observations: {}'.format(df.shape[0]))
print('# features:     {}'.format(df.shape[1]-1))

In [ ]:
# Display first 5 rows
df.head()

In [ ]:
# Print a concise summary of a DataFrame
df.info()

In [ ]:
# Generate descriptive statistics
df.describe()

In [ ]:
# The column labels of the DataFrame.
df.columns

In [ ]:
# Check for missing values
df.isnull().sum()

In [ ]:
# check for unique values in column bank_account
df['bank_account'].unique()

In [ ]:
# Checking for data imbalance
df['bank_account'].value_counts()

We see that we have NaNs in the column bank account. Our goal is to fill this Nan values with values that our model (hopefully) predicts right.

What we need to do is:
* Create a data frame without NaNs. This will be the data we than will split into train and test data.
* Create a data frame with all the NaN values. This will be the data we than will have our model predict with.

In [ ]:
# Create a new data frame without the NaN in our target feature
df_wo_target_nan = df.dropna()

# set new index for our dataframe without the NaNs
df_wo_target_nan = df_wo_target_nan.reset_index(drop=True)
df_wo_target_nan.isnull().value_counts()
df_wo_target_nan.head()

In [ ]:
# create a new data frame with only the NaN in our target feature
df_with_target_nan = df.where(df['bank_account'].isnull())
df_with_target_nan.head()

# Set a new index for our dataframe with the NaNs
df_with_target_nan = df_with_target_nan.reset_index(drop=True)
df_with_target_nan.head()

At this point we now focus on our dataframe without the NaNs. 
Quick look at the dataframe:

In [ ]:
df_wo_target_nan.info()

In [ ]:
df_wo_target_nan.describe()

In [ ]:
df_wo_target_nan.head().T

In [ ]:
# Show overview of all the unique values of the dataframe:
for column in df_wo_target_nan.columns:
    unique_values = df_wo_target_nan[column].unique()
    print(f"Column '{column}' has {len(unique_values)} unique value(s):")
    print(unique_values)
    print()

In [ ]:
# Checking for duplicate values
print(f"duplicate values in columns")

display(df_wo_target_nan.duplicated().value_counts())

print('No duplicates found.')
print("______"*30)

In [ ]:
# Plot distribution of features 

features = df_wo_target_nan.columns.tolist()
features.remove('bank_account')

fig,ax = plt.subplots(4,3,figsize=(34,30))
count = 0
for item in features:
    sns.histplot(df_wo_target_nan[item], kde=True, ax=ax[int(count/3)][count%3], color='#33658A').set(title=item, xlabel='')
    count += 1
ax.flat[-1].set_visible(False)
fig.tight_layout(pad=3);

In [ ]:
# Plotting correlation between numeric columns
numeric_df = df_wo_target_nan.select_dtypes(include='number')
correlation_matrix = numeric_df.corr()

fig = go.Figure(data=go.Heatmap(z=correlation_matrix.values, x=correlation_matrix.columns, y=correlation_matrix.index))

fig.show()

In [ ]:
# Initialize an empty matrix to store Cramér's V 
'''
n_columns = len(df_wo_target_nan.columns)
cramers_matrix = np.zeros((n_columns, n_columns))

# Iterate over each pair of columns
for i in range(n_columns):
    for j in range(n_columns):
        # Create a contingency table for the column pair
        contingency_table = pd.crosstab(df_wo_target_nan.iloc[:, i], df.iloc[:, j])
        
        # Perform the chi-square test and calculate Cramér's V
        chi2, _, _, _ = chi2_contingency(contingency_table)
        n = len(df_wo_target_nan)
        cramers_v = np.sqrt(chi2 / (n * (min(contingency_table.shape) - 1)))
        
        # Store the Cramér's V value in the matrix
        cramers_matrix[i, j] = cramers_v

# Create a DataFrame from the matrix with column names as indices and columns
cramers_df = pd.DataFrame(cramers_matrix, index=df_wo_target_nan.columns, columns=df_wo_target_nan.columns)

# Create a heatmap using Seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(cramers_df, annot=True, fmt=".2f", cmap="crest", square=True)
plt.title("Cramér's V Heatmap")
plt.show()
'''

In [ ]:
# descriptive statistics for specific columns with numerical features
num_cols = ["age_of_respondent", "household_size", "year"]
# check table again
num_data = df_wo_target_nan[num_cols]
print("______"*30)
display(num_data.head(10))
print("______"*30)
display(num_data.describe())
print("______"*30)

75% of the respondents have 49 years old. This could be explained by the fact that the average life expectancy in Africa is still very low compared to other continents. Concerning this column, there is also a big difference comparing with the maximum value of 100. There are some ouliers in these column, that must be analysed before modelling.
<br>

We can also predict outliers in the column ‘household_size’ due to the huge difference between the 75% of the respondents having 5 people living in their houses, and a maximum value of 21 people.

In [ ]:
df = px.data.tips()
fig = px.box(df_wo_target_nan, y='age_of_respondent')
fig.show()

In [ ]:
df = px.data.tips()
fig = px.box(df_wo_target_nan, y='household_size')
fig.show()

Find out if there is a relation between the features (which contains objects) and our target feature, using the "Cramers V".

* Small Effect:
Cramér's V values close to 0 indicate a weak or negligible association between the categorical variables.

* Medium Effect:
Cramér's V values around 0.1 to 0.3 suggest a moderate association. This indicates that the variables have some degree of dependency, but the association may not be very strong.

* Large Effect:
Cramér's V values close to 0.3 or higher indicate a relatively strong association between the categorical variables. This suggests a notable dependency or relationship between the variables.


In [ ]:
# Check each column vs. the target column if there is a correlation by creating a function using the Cramér's V:

# make a list with each column name 
column_names = df_wo_target_nan.columns.tolist()
# delete bank_account from the list
column_names.remove('bank_account')
# create target value
target_column = 'bank_account'

def cramers_v(list, target_column):
    
    for name in list:
        # Create a contingency table
        contingency_table = pd.crosstab(df_wo_target_nan[name], df_wo_target_nan[target_column])

        # Perform chi-square test
        chi2, p, *_ = chi2_contingency(contingency_table)

        # Calculate Cramér's V
        n = len(df_wo_target_nan)
        cramers_v = np.sqrt(chi2 / (n * (min(contingency_table.shape) - 1)))

        # only print output is cramers_v is bigger than 0.1
        if cramers_v >= 0.1:

            print('-----------------------------')
            print(f'{name} vs. {target_column}')
            # print("Chi-square:", chi2)
            # print("p-value:", p)
            print("Cramér's V:", round(cramers_v, 3))
        

cramers_v(column_names, target_column)
cramers_v(['job_type'], 'education_level')

Because there is a medium correlation between job type and education level and a high correlation between job type/bank account and education level/bank account: let's make a plot to show it.


In [ ]:
# Check each column vs. the target column if there is a correlation by creating a function using the Cramér's V:

# make a list with each column name 
column_names = df_wo_target_nan.columns.tolist()
# delete bank_account from the list
column_names.remove('bank_account')
column_names.remove('uniqueid')
# create target value
target_names = column_names

def cramers_v(list_columns, list_targets):
    
    for name in list_columns:
        # Create a contingency 
        for target in list_targets:
            if name != target:
                contingency_table = pd.crosstab(df_wo_target_nan[name], df_wo_target_nan[target])

                # Perform chi-square test
                chi2, p, *_ = chi2_contingency(contingency_table)

                # Calculate Cramér's V
                n = len(df_wo_target_nan)
                cramers_v = np.sqrt(chi2 / (n * (min(contingency_table.shape) - 1)))

                # only print output is cramers_v is bigger than 0.1
                if cramers_v >= 0.3:

                    print('-----------------------------')
                    print(f'{name} vs. {target}')
                    # print("Chi-square:", chi2)
                    # print("p-value:", p)
                    print("Cramér's V:", round(cramers_v, 3))
            

cramers_v(column_names, target_names)
# cramers_v(['job_type'], 'education_level')

In [ ]:
# Countplot with bank account, hue=gender
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.countplot(x=df_wo_target_nan['bank_account'], hue=df_wo_target_nan['gender_of_respondent']);

## Cleaning the data

We now drop columns if they:

* are an ID
* have no to negligible correlation to the target feature


In [ ]:
df_wo_target_nan.columns

Renaming the columns for better readability:

In [ ]:
# Renaming the column names
df_wo_target_nan.rename(columns = {'country': 'country',
        'year': 'year',
        'uniqueid': 'id',
        'location_type': 'location',
        'cellphone_access': 'cellphone',
        'household_size': 'household_size',
        'age_of_respondent': 'age',
        'gender_of_respondent': 'gender',
        'relationship_with_head': 'relationship_with_head', 
        'marital_status': 'marital_status', 
        'education_level': 'education',
        'job_type': 'job',
        'bank_account': 'bank_account'},
        inplace = True)

df_wo_target_nan.head().T

In [ ]:
# drop id column
df_wo_target_nan = df_wo_target_nan.drop('id', axis=1)
df_wo_target_nan.head().T

In [ ]:
df_wo_target_nan.nunique()

In [ ]:
# Plotting the target variable
plt.title('Bank Account Count')
sns.countplot(x=df_wo_target_nan.bank_account);

In [ ]:
# plot which shows percentage of people with and without a bank account
data = df_wo_target_nan['bank_account']

plt.hist(data, weights=np.ones(len(data)) / len(data))

plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.show()

In [ ]:
df_wo_target_nan['bank_account'].value_counts()

In the two histogramm plots we can see that the data isn't well balanced. Out of 23.524 people in our dataset only 3.312 people (~18%) have a bank account. 20.212 don't.

&rarr; We need to remove our majority class!
<br>

In this plot we can see that we have outliers in the household_size column and also in the age column.

In [ ]:
# Plotting a pairplot to see how the variables differ depending on our target variable - 'bank_account'
sns.pairplot(df_wo_target_nan, hue='bank_account', height=2);

---

<br>

We have many categorical values and a few discreet values:
* create dummy variables (hot-one encoding for categorical values)
* bin discreet values (discreet values)
* create a csv file with the results


In [2]:
df_final = pd.read_csv('data/data_binary.csv')

In [3]:
# make values astyp int to properly use the RandomOverSampler
df_final = df_final.astype(int)

In [4]:
df_final.head()

,household_size_1-2,household_size_2-5,household_size_5-10,household_size_10-25,age_0-20,age_20-40,age_40-70,age_70-100,country_Rwanda,country_Tanzania,...,job_Farming and Fishing,job_Formally employed Government,job_Formally employed Private,job_Government Dependent,job_Informally employed,job_No Income,job_Other Income,job_Remittance Dependent,job_Self employed,bank_account_Yes
0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,0,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,0,0,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


Because our target feature is unbalanced: we have an overfitting in our target category &rarr; no is the majority class 

In [5]:
# TRAIN TEST SPLIT  
# Defining X and y
features = df_final.columns.tolist()
features.remove('bank_account_Yes')
# Target
# y = heart.heart_attack
# Predictors
# X = heart.drop('heart_attack', axis=1) instead of making a list of features in line 2 and 3. So you don't have to drop the column in the dataframe

X = df_final[features]
y = df_final.bank_account_Yes

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True, stratify=y)

# Check the shape of the data sets
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

X_train: (16466, 39)
y_train: (16466,)
X_test: (7058, 39)
y_test: (7058,)


Use oversampling to balance out the minority class

In [6]:
# Oversampling
oversample = imblearn.over_sampling.RandomOverSampler(sampling_strategy='minority')

X_train_over, y_train_over = oversample.fit_resample(X_train, y_train)

display('X_train: ' + str(X_train.shape))
display('y_train: ' + str(y_train.shape))
display('X_train_over: ' + str(X_train_over.shape))
display('y_train_over: ' + str(y_train_over.shape))

count_yes, count_no = 0, 0
for i in y_train_over:
    if i == 1:
        count_yes += 1
    if i == 0:
        count_no += 1
print (count_yes, count_no)


'X_train: (16466, 39)'

'y_train: (16466,)'

'X_train_over: (28296, 39)'

'y_train_over: (28296,)'

14148 14148


# Modeling

## Decision Tree

In [7]:
dtree = DecisionTreeClassifier()
dtree.fit(X_train_over, y_train_over)

DecisionTreeClassifier()

In [9]:
# Your code here!
y_pred_tree = dtree.predict(X_test)
print(confusion_matrix(y_test, y_pred_tree))
print(classification_report(y_test, y_pred_tree))

[[4964 1100]
 [ 362  632]]
              precision    recall  f1-score   support

           0       0.93      0.82      0.87      6064
           1       0.36      0.64      0.46       994

    accuracy                           0.79      7058
   macro avg       0.65      0.73      0.67      7058
weighted avg       0.85      0.79      0.81      7058



## Logistic Regression

In [10]:
# Logistic Regression (base?) - line model:
#df = pd.read_csv('data/data.csv')
# create a new data frame without the NaN in our target feature
#df_wo_target_nan = df.dropna(axis=0)
# Defining X and y
#features = df_wo_target_nan.columns.tolist()
#features.remove('bank_account')
#X = df_wo_target_nan[features]
#y = df_wo_target_nan.bank_account
# Splitting the dataset
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, shuffle=True)
#Oversampling
#oversample = imblearn.over_sampling.RandomOverSampler(sampling_strategy='minority')
#X_train_over, y_train_over = oversample.fit_resample(X_train, y_train)
# Encode categorical features
enc = preprocessing.OrdinalEncoder()
X_train_fitted_transformed = enc.fit_transform(X_train_over)
X_test_fitted_transformed = enc.fit_transform(X_test)
### Using Logistic regression using non-scaled data
log_reg = LogisticRegression(max_iter=10000)
log_reg.fit(X_train_fitted_transformed, y_train_over)
y_pred_train = log_reg.predict(X_train_fitted_transformed)
y_pred = log_reg.predict(X_test_fitted_transformed)
# Print accuracy of our model
print("Accuracy on train set:", round(accuracy_score(y_train_over, y_pred_train), 2))
print("Accuracy on test set:", round(accuracy_score(y_test, y_pred), 2))
print("--------"*10)
# Print MAE of our model
y_pred_train_ = []
y_pred_ = []
for i in y_pred_train:
    if i == 1: y_pred_train_.append(1)
    if i == 0: y_pred_train_.append(0)
for i in y_pred:
    if i == 1: y_pred_.append(1)
    if i == 0: y_pred_.append(0)
print("Mean absolute error on train set:", round(mean_absolute_error(y_train_over, y_pred_train_),2))
print("Mean absolute error on test set:", round(mean_absolute_error(y_test, y_pred_),2))
print("--------"*10)

Accuracy on train set: 0.77
Accuracy on test set: 0.79
--------------------------------------------------------------------------------
Mean absolute error on train set: 0.23
Mean absolute error on test set: 0.21
--------------------------------------------------------------------------------


## Random Forest Model

In [11]:
# initializing the model and fitting it
forest = RandomForestClassifier()
forest.fit(X_train_over, y_train_over)

RandomForestClassifier()

In [12]:
# Evaluating the model
y_pred_forest = forest.predict(X_test)
print(confusion_matrix(y_test, y_pred_forest))
print(classification_report(y_test, y_pred_forest))

[[4992 1072]
 [ 355  639]]
              precision    recall  f1-score   support

           0       0.93      0.82      0.87      6064
           1       0.37      0.64      0.47       994

    accuracy                           0.80      7058
   macro avg       0.65      0.73      0.67      7058
weighted avg       0.85      0.80      0.82      7058



In [13]:
RSEED = 50
# Hyperparameter grid
param_grid = {
    'n_estimators': np.linspace(10, 200).astype(int),
    'max_depth': [None] + list(np.linspace(3, 20).astype(int)),
    'max_features': ['auto', 'sqrt', None] + list(np.arange(0.5, 1, 0.1)),
    'max_leaf_nodes': [None] + list(np.linspace(10, 50, 500).astype(int)),
    'min_samples_split': [2, 5, 10],
    'bootstrap': [True, False]
}

# Estimator for use in random search
estimator = RandomForestClassifier(random_state = RSEED)
# pimp the scoring for rs
from sklearn.metrics import fbeta_score, make_scorer
scorer = make_scorer(fbeta_score, beta=2)
# Create the random search model
rs = RandomizedSearchCV(estimator, param_grid, n_jobs = -1, 
                        scoring = scorer, cv = 3, 
                        n_iter = 10, verbose = 5, random_state=RSEED)

# Fit 
rs.fit(X_train_over, y_train_over)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/Users/shahiwinderling/neuefische/Financial-Inclusion-Project/.venv/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/shahiwinderling/neuefische/Financial-Inclusion-Project/.venv/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/shahiwinderling/neuefische/Financial-Inclusion-Project/.venv/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated

[CV 1/3] END bootstrap=False, max_depth=10, max_features=auto, max_leaf_nodes=28, min_samples_split=10, n_estimators=87;, score=0.750 total time=   0.9s
[CV 3/3] END bootstrap=False, max_depth=10, max_features=auto, max_leaf_nodes=28, min_samples_split=10, n_estimators=87;, score=0.760 total time=   0.8s
[CV 2/3] END bootstrap=False, max_depth=10, max_features=auto, max_leaf_nodes=28, min_samples_split=10, n_estimators=87;, score=0.741 total time=   0.8s
[CV 2/3] END bootstrap=False, max_depth=7, max_features=0.7, max_leaf_nodes=26, min_samples_split=10, n_estimators=153;, score=0.748 total time=   3.4s
[CV 1/3] END bootstrap=False, max_depth=7, max_features=0.7, max_leaf_nodes=26, min_samples_split=10, n_estimators=153;, score=0.780 total time=   3.6s


/Users/shahiwinderling/neuefische/Financial-Inclusion-Project/.venv/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV 1/3] END bootstrap=True, max_depth=16, max_features=auto, max_leaf_nodes=43, min_samples_split=5, n_estimators=29;, score=0.757 total time=   0.3s
[CV 1/3] END bootstrap=True, max_depth=12, max_features=0.7999999999999999, max_leaf_nodes=33, min_samples_split=2, n_estimators=188;, score=0.769 total time=   4.1s
[CV 2/3] END bootstrap=True, max_depth=12, max_features=0.7999999999999999, max_leaf_nodes=33, min_samples_split=2, n_estimators=188;, score=0.752 total time=   4.1s


/Users/shahiwinderling/neuefische/Financial-Inclusion-Project/.venv/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/shahiwinderling/neuefische/Financial-Inclusion-Project/.venv/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/shahiwinderling/neuefische/Financial-Inclusion-Project/.venv/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated

[CV 3/3] END bootstrap=False, max_depth=7, max_features=0.7, max_leaf_nodes=26, min_samples_split=10, n_estimators=153;, score=0.792 total time=   3.6s
[CV 2/3] END bootstrap=True, max_depth=16, max_features=auto, max_leaf_nodes=43, min_samples_split=5, n_estimators=29;, score=0.760 total time=   0.3s
[CV 3/3] END bootstrap=True, max_depth=12, max_features=0.7999999999999999, max_leaf_nodes=33, min_samples_split=2, n_estimators=188;, score=0.780 total time=   4.4s
[CV 1/3] END bootstrap=True, max_depth=4, max_features=auto, max_leaf_nodes=17, min_samples_split=5, n_estimators=25;, score=0.711 total time=   0.2s
[CV 3/3] END bootstrap=True, max_depth=16, max_features=auto, max_leaf_nodes=43, min_samples_split=5, n_estimators=29;, score=0.763 total time=   0.2s
[CV 2/3] END bootstrap=True, max_depth=4, max_features=auto, max_leaf_nodes=17, min_samples_split=5, n_estimators=25;, score=0.700 total time=   0.2s


/Users/shahiwinderling/neuefische/Financial-Inclusion-Project/.venv/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/shahiwinderling/neuefische/Financial-Inclusion-Project/.venv/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV 3/3] END bootstrap=True, max_depth=4, max_features=auto, max_leaf_nodes=17, min_samples_split=5, n_estimators=25;, score=0.740 total time=   0.2s
[CV 1/3] END bootstrap=True, max_depth=16, max_features=0.7999999999999999, max_leaf_nodes=11, min_samples_split=5, n_estimators=25;, score=0.712 total time=   0.4s
[CV 2/3] END bootstrap=True, max_depth=16, max_features=0.7999999999999999, max_leaf_nodes=11, min_samples_split=5, n_estimators=25;, score=0.658 total time=   0.4s
[CV 3/3] END bootstrap=True, max_depth=16, max_features=0.7999999999999999, max_leaf_nodes=11, min_samples_split=5, n_estimators=25;, score=0.754 total time=   0.4s
[CV 2/3] END bootstrap=True, max_depth=8, max_features=0.6, max_leaf_nodes=22, min_samples_split=5, n_estimators=83;, score=0.721 total time=   1.2s
[CV 1/3] END bootstrap=True, max_depth=8, max_features=0.6, max_leaf_nodes=22, min_samples_split=5, n_estimators=83;, score=0.755 total time=   1.3s
[CV 2/3] END bootstrap=False, max_depth=14, max_features=

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=50),
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 3, 3, 3, 4, 4, 4, 5,
                                                      5, 5, 6, 6, 6, 7, 7, 7, 8,
                                                      8, 8, 9, 9, 9, 10, 10, 10,
                                                      11, 11, 12, 12, 12, ...],
                                        'max_features': ['auto', 'sqrt', None,
                                                         0.5, 0.6, 0.7,
                                                         0.7999999999999999,
                                                         0.8999999999999999],
                                        'max_leaf_nodes': [None, 10, 10, 10, 10,
                                                           10, 10, 10, 1...
                                                           10, 10, 10, 10, 11,
                                                           11, 11, 11, 11, 11,
                                                           11, 11, 11, 11, 11,
                                                           11, 12, 12, 12, 12, ...],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': array([ 10,  13,  17,  21,  25,  29,  33,  37,  41,  44,  48,  52,  56,
        60,  64,  68,  72,  75,  79,  83,  87,  91,  95,  99, 103, 106,
       110, 114, 118, 122, 126, 130, 134, 137, 141, 145, 149, 153, 157,
       161, 165, 168, 172, 176, 180, 184, 188, 192, 196, 200])},
                   random_state=50, scoring=make_scorer(fbeta_score, beta=2),
                   verbose=5)

In [14]:
rs.best_params_

{'n_estimators': 153,
 'min_samples_split': 10,
 'max_leaf_nodes': 26,
 'max_features': 0.7,
 'max_depth': 7,
 'bootstrap': False}

In [15]:
y_pred_forest = rs.predict(X_test)
print(confusion_matrix(y_test, y_pred_forest))
print(classification_report(y_test, y_pred_forest))

[[4307 1757]
 [ 214  780]]
              precision    recall  f1-score   support

           0       0.95      0.71      0.81      6064
           1       0.31      0.78      0.44       994

    accuracy                           0.72      7058
   macro avg       0.63      0.75      0.63      7058
weighted avg       0.86      0.72      0.76      7058



| True Positives | True Negatives | Accuracy |
|----------------|----------------|----------|
|725 | 4923 | 0.80 |

### Tweaking Random Forest model with best_params

In [16]:
# initializing the model with best params and fitting it
forest = RandomForestClassifier(n_estimators=79, min_samples_split=10, max_leaf_nodes=32, max_features=0.7, max_depth=13, bootstrap=False)
forest.fit(X_train_over, y_train_over)

RandomForestClassifier(bootstrap=False, max_depth=13, max_features=0.7,
                       max_leaf_nodes=32, min_samples_split=10,
                       n_estimators=79)

In [17]:
# Evaluating the model
y_pred_forest = forest.predict(X_test)
print(confusion_matrix(y_test, y_pred_forest))
print(classification_report(y_test, y_pred_forest))

[[4652 1412]
 [ 235  759]]
              precision    recall  f1-score   support

           0       0.95      0.77      0.85      6064
           1       0.35      0.76      0.48       994

    accuracy                           0.77      7058
   macro avg       0.65      0.77      0.66      7058
weighted avg       0.87      0.77      0.80      7058



| True Positives | True Negatives | Accuracy | 
|----------------|----------------|----------|
|734 | 4851 | 0.79 |

<br>

## KNN Model

In [18]:
# Separate the features (X) and the target variable (y)
#X = df.drop('bank_account_Yes', axis=1) 
#y = df['bank_account_Yes']

# Split the data into train and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Perform oversampling on the minority class
oversampler = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

# Perform undersampling on the majority class
undersampler = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train_resampled, y_train_resampled)

# Define the parameter grid
param_grid = {'n_neighbors': [3, 5, 7, 9, 11, 15, 20], 'weights': ['uniform', 'distance']}

# Create and train the KNN model with hyperparameter tuning
knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best hyperparameters and corresponding score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Hyperparameters:", best_params)
print("Best Score:", best_score)

# Train the KNN model with the best hyperparameters
best_knn = KNeighborsClassifier(**best_params)
best_knn.fit(X_train_resampled, y_train_resampled)

# Predict on the test set
y_pred = best_knn.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

Best Hyperparameters: {'n_neighbors': 9, 'weights': 'distance'}
Best Score: 0.8269721267039525
              precision    recall  f1-score   support

           0       0.93      0.80      0.86      6064
           1       0.33      0.62      0.43       994

    accuracy                           0.77      7058
   macro avg       0.63      0.71      0.65      7058
weighted avg       0.84      0.77      0.80      7058

